In [2]:
!pip install spacy==3.0.0

     |████████████████████████████████| 12.7 MB 222 kB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 628 kB 43.2 MB/s 
     |████████████████████████████████| 9.1 MB 11.9 MB/s 
     |████████████████████████████████| 451 kB 40.9 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [4]:
#import the packages
import pandas as pd
import random
import operator


import spacy
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL
from spacy.training import Example
from spacy.util import minibatch, compounding

In [5]:
#Read input file
input_file= pd.read_csv("/content/drive/MyDrive/Doc_classification/train_test_article_data.csv")

In [6]:
# Meta data of the input file
meta_Data=input_file.info()

#exploded input file based on category_ids for getting unique values for label adding in TextCategorizer
explode_category=(input_file.set_index("text")["category_ids"]
   .str.split(',')
   .explode()
   .rename("category_ids")
   .reset_index())


Categories=list(explode_category["category_ids"].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15999 entries, 0 to 15998
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             15999 non-null  object
 1   category_ids     15999 non-null  object
 2   category_scores  15999 non-null  object
dtypes: object(3)
memory usage: 375.1+ KB


In [17]:
# splitting train_test data
dataset_size=len(input_file)/10
print(int(dataset_size*2))
train_data=input_file[:int(dataset_size*2)]
#test_data=input_file.text[]

3199


In [9]:
# function to preprocess-remove stop words from the input text
def remove_Stopwords(text):
    doc=nlp(text)
    stoprm_text=" ".join([str(token) for token in doc if token.is_stop==False])
    return stoprm_text

In [10]:
# Formatting training data before training
def train_dataformatting(train_data):
    
    train_formatted_tup=[]
    for index,row in train_data.iterrows():
        text=row["text"]
        labels=row["category_ids"].split(',')
        label_score=row["category_scores"].split(',')
        label_dic={}
        for label in range(len(labels)):
            label_dic[labels[label]]=label_score[label]
        train_formatted_tup.append((text,{"cats":label_dic}))
    return train_formatted_tup


In [18]:
# formatted train data
Formatted_traindata=train_dataformatting(train_data)

In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.7 MB 73 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
#nlp = spacy.load("en_core_web_md")
#nlp = spacy.load("en_core_web_sm")

In [12]:
# creating blank spacy model
nlp = spacy.blank("en")

In [13]:
# model configuration
config = {"threshold": 0.5,"model": DEFAULT_MULTI_TEXTCAT_MODEL}

textcat = nlp.add_pipe("textcat_multilabel", config=config)

In [14]:
# adding labels into modelling-label
for label in Categories:
   textcat.add_label(label) 

In [19]:
# training initiation
train_examples = [Example.from_dict(nlp.make_doc(text), label) for text,label in Formatted_traindata]

textcat.initialize(lambda: train_examples, nlp=nlp)

In [20]:
# model training 
epochs=3
with nlp.select_pipes(enable="textcat_multilabel"):
    optimizer =nlp.resume_training()
    batches = minibatch(Formatted_traindata, size=compounding(4., 32., 1.001))
    for i in range(epochs):
      losses = {}
      for batch in batches:
          for text, label in batch:
              text=remove_Stopwords(text)
              print(text,label)
              doc = nlp.make_doc(text)
              example = Example.from_dict(doc, label)
              nlp.update([example], sgd=optimizer, drop=0.2,losses=losses)

Cosby Release Means Victims Workplace Sexual Harassment technicalities Cosby release hardly consolation victim sexual misconduct thinking taking legal action response . . {'cats': {'135': '1', ' 96': ' 1', ' 132': ' 1', ' 1654': ' 1', ' 1651': ' 1', ' 121': ' 0.8471', ' 2': ' 0.8424', ' 127': ' 0.8035', ' 125': ' 0.7742', ' 98': ' 0.7655'}}
State high court upholds ending adult trials youths California Supreme Court upheld state 2018 law barring 14- 15 - year - olds tried adults sent adult prisons crimes murder , arson , robbery , rape kidnapping . {'cats': {'135': '1', ' 132': ' 1', ' 151': ' 1', ' 96': ' 1', ' 1654': ' 1', ' 1651': ' 1', ' 121': ' 1', ' 118': ' 0.8472', ' 2': ' 0.8335', ' 120': ' 0.7598'}}
Bill Cosby : released , happens – line payout ? Actor served years 10 - year prison sentence .. {'cats': {'132': '1', ' 1651': ' 1', ' 96': ' 1', ' 135': ' 0.9803', ' 121': ' 0.9563', ' 1654': ' 0.917', ' 2': ' 0.7936', ' 127': ' 0.7668', ' 125': ' 0.7551', ' 128': ' 0.7062'}}
Kari

In [21]:
# input for model prediction
testdata=test_data=input_file.text[11000]
print(testdata)


Rare ancient scroll found in Israel Cave of Horror The verses are thought to have been written by Jewish rebels hiding in the cave from Romans. .


In [34]:
# prediction
doc2 = nlp(testdata)
article_Categories=doc2.cats


In [36]:
#sorted article label based on probability
sorted_d = dict( sorted(article_Categories.items(), key=operator.itemgetter(1),reverse=True))
print(sorted(article_Categories, reverse=True)) 

['996', '989', '98', '96', '958', '956', '940', '927', '915', '9', '893', '890', '886', '884', '88', '875', '87', '86', '859', '856', '854', '847', '84', '832', '83', '825', '817', '816', '814', '813', '812', '81', '809', '806', '803', '800', '80', '799', '797', '792', '791', '786', '785', '784', '782', '778', '776', '775', '774', '770', '77', '761', '758', '757', '753', '751', '750', '747', '745', '744', '734', '732', '731', '730', '728', '727', '726', '724', '722', '721', '719', '718', '715', '714', '713', '712', '71', '70', '7', '692', '686', '684', '68', '672', '67', '667', '665', '660', '66', '652', '647', '643', '641', '637', '63', '623', '618', '614', '613', '610', '61', '608', '605', '604', '601', '60', '6', '596', '592', '59', '58', '577', '575', '573', '570', '569', '568', '567', '566', '563', '562', '561', '560', '559', '557', '556', '555', '554', '545', '544', '543', '541', '54', '539', '537', '53', '527', '52', '518', '515', '51', '50', '5', '486', '485', '48', '478', '473

In [41]:
# predicton output
print (testdata)
for k,v in sorted(article_Categories.items(), key=operator.itemgetter(1),reverse=True)[:10]:
  print (k,":",v)

Rare ancient scroll found in Israel Cave of Horror The verses are thought to have been written by Jewish rebels hiding in the cave from Romans. .
1654 : 0.8850257992744446
1658 : 0.872995913028717
1651 : 0.851193904876709
9 : 0.8396334052085876
151 : 0.8288823962211609
16 : 0.8092415928840637
1323 : 0.8021531701087952
1693 : 0.8012615442276001
96 : 0.7994937300682068
125 : 0.7872372269630432
